In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import shutil
import tensorflow as tf

from keras.layers.advanced_activations import LeakyReLU, PReLU
from math import cos, sin, pi
from PIL import Image
from tqdm import tqdm
from tensorflow.keras import Sequential, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionResNetV2

In [2]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [3]:
# 경로 이동
os.chdir('data')

In [4]:
os.listdir()

['aug_rotation_noise_upsidedown.csv',
 'sample_submission.csv',
 'train_imgs.zip',
 'baseline_submission.csv',
 'baseline_with_augmentation.h5',
 'train_df.csv',
 'train_imgs',
 'test_imgs',
 'test_imgs.zip',
 '.ipynb_checkpoints']

In [5]:
# train 데이터 중 10%를 검증 데이터로 사용

# csv 파일 불러오기
data = pd.read_csv('train_df.csv')
submission = pd.read_csv('sample_submission.csv')

# 경로 설정
data_paths = sorted(glob.glob('./train_imgs/*.jpg'))
test_paths = sorted(glob.glob('./test_imgs/*.jpg'))

data['path'] = data_paths

In [6]:
# 데이터 프레임 랜덤하게 분할

# 전체 데이터 중 90%는 학습 데이터 활용
train = data.sample(frac=0.9, random_state=2021)
print('학습 데이터 길이는: ', len(train))

# 전체 데이터 중 10%는 검증 데이터 활용
valid = data.drop(train.index)
print('검증 데이터 길이는: ', len(valid))

학습 데이터 길이는:  3776
검증 데이터 길이는:  419


In [7]:
def trainGenerator():
    for i in range(len(train)):
        img = tf.io.read_file(train['path'][i])  # path(경로)를 통해 이미지 읽기
        # 경로를 통해 불러온 이미지를 tensor로 변환
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [270, 480])  # 이미지 resize
        img = img/255
        target = train.iloc[:, 1:49].iloc[i, :]  # keypoint 뽑아주기
        target = target/4

        yield (img, target)

In [8]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 32

train_dataset = tf.data.Dataset.from_generator(
    trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([270, 480, 3]), tf.TensorShape([48])))
train_dataset = train_dataset.batch(batch_size).prefetch(AUTOTUNE)
valid_dataset = tf.data.Dataset.from_generator(
    trainGenerator, (tf.float32, tf.float32), (tf.TensorShape([270, 480, 3]), tf.TensorShape([48])))
valid_dataset = valid_dataset.batch(batch_size).prefetch(AUTOTUNE)

In [9]:
train.reset_index(drop=True, inplace=True)
valid.reset_index(drop=True, inplace=True)

In [10]:
# Callback 설정
early_stopping = EarlyStopping(patience=3)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    patience=2,
    factor=0.85,
    min_lr=1e-7,
    verbose=1
)

model_checkpoint_callback = ModelCheckpoint(  # 에포크마다 현재 가중치를 저장
    filepath="./model_checkpoint_callback_210318_{epoch}.h5",  # 모델 파일 경로
    monitor='val_loss',  # val_loss가 좋아지지 않으면 모델 파일을 덮어쓰지 않음.
    save_best_only=True
)

callbacks = [early_stopping, reduce_lr, model_checkpoint_callback]

In [11]:
with strategy.scope():

    base_model = InceptionResNetV2(input_shape=(
        270, 480, 3), include_top=False, weights='imagenet')

    x = base_model.output
    x = Flatten()(x)
    x = Dense(512, activation='relu', input_dim=(7*13*1536))(x)
    x = Dropout(0.1)(x)
    predictions = Dense(48)(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    model.compile(optimizer=Adam(learning_rate=0.01),
                  loss='mean_squared_error',
                  metrics=['accuracy'])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 270, 480, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 134, 239, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 134, 239, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 134, 239, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [13]:
history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=valid_dataset,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
INFO:tensorflow:batch_all_reduce: 492 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 492 all-reduces with algorithm = nccl, num_packs = 1
118/118 [==============================] - 295s 2s/step - loss: 127501.1447 - accuracy: 0.0603 - val_loss: 1248625426432.0000 - val_accuracy: 0.1353
Epoch 2/100
118/118 [==============================] - 196s 2s/step - loss: 3528.9276 - accuracy: 0.0951 - val_loss: 17507.7832 - val_accuracy: 0.0101
Epoch 3/100
118/118 [==============================] - 196s 2s/step - loss: 3309.0103 - accuracy: 0.1354 - val_loss: 1178.4860 - val_accuracy: 0.1713
Epoch 4/100
118/118 [==============================] - 194s 2s/step - loss: 3214.5336 - accuracy: 0.1130 - val_loss: 1138.6763 - val_accuracy: 0.0156
Epoch 5/100
118/118 [==============================] - 195s 2s/step - loss: 3153.3193 - accuracy: 0.1222 - val_loss: 998.5696 - val_accuracy: 0.0358
Epoch 6/100
118/118 [==============================] - 198s 2s/st

In [14]:
model.save('model_210318.h5')

In [15]:
X_test=[]

for test_path in tqdm(test_paths):
    img=tf.io.read_file(test_path)
    img=tf.image.decode_jpeg(img, channels=3)
    img=tf.image.resize(img, [270,480])
    img=img/255
    X_test.append(img)

X_test=tf.stack(X_test, axis=0)
X_test.shape

100%|██████████| 1600/1600 [00:15<00:00, 100.20it/s]


ResourceExhaustedError: OOM when allocating tensor with shape[1600,270,480,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Pack] name: stack

In [ ]:
pred = model.predict(X_test)

In [ ]:
submission = pd.read_csv('./sample_submission.csv')
# image size를 1920x1080 -> 480x270으로 바꿔서 예측했으므로 * 4
submission.iloc[:, 1:] = pred*4

In [ ]:
# submission
submission.to_csv('submission_210318_01.csv', index=False)